In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Conv2DTranspose, AveragePooling2D, Add
from scipy.interpolate import RectBivariateSpline
import tensorflow.contrib.eager as tfe
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Boundary import Boundary1D
from collections.abc import Iterable
import itertools, h5py
from multiprocessing import Pool as ThreadPool
opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.95)
conf = tf.ConfigProto(gpu_options=opts)
tfe.enable_eager_execution(config=conf)

In [316]:
class Upsample(tf.keras.layers.Layer):
    def __init__(self, upsample_ratio = (2,2), output_dtype = tf.float32, data_format = 'channels_first', resize_method = tf.image.ResizeMethod.BICUBIC, **kwargs):
        super(Upsample, self).__init__(**kwargs)
        self.output_dtype = output_dtype
        self.data_format = data_format
        self.resize_method = resize_method
        if not isinstance(upsample_ratio, Iterable):
            self.upsample_ratio = (upsample_ratio, upsample_ratio)
        else:
            self.upsample_ratio = upsample_ratio
            
    def build(self, input_shape):
        super(Upsample, self).build(input_shape)
        
    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            return tf.TensorShape(list(input_shape)[0:2] + [self.upsample_ratio[0] * input_shape[-2], self.upsample_ratio[1] * input_shape[-1]])
        else:
            return tf.TensorShape([input_shape[0], self.upsample_ratio[0] * input_shape[-3], self.upsample_ratio[1] * input_shape[-2], input_shape[-1]])
    
#     def upsample(self, inp):
#         dummy_coords_inp = (np.linspace(0,1,int(inp.shape[-2])), np.linspace(0,1,int(inp.shape[-1])))
#         dummy_coords_out = (np.linspace(0,1,int(self.compute_output_shape(inp.shape)[-2])),np.linspace(0,1,int(self.compute_output_shape(inp.shape)[-1])))
#         #pdb.set_trace()
#         res = []
#         for i in range(inp.shape[0]):
#             spl = RectBivariateSpline(dummy_coords_inp[0], dummy_coords_inp[1], np.array(inp[i,...]))
#             res.append(spl(dummy_coords_out[0], dummy_coords_out[1]))
#         return tf.dtypes.cast(np.array(res), dtype=self.output_dtype)
    
#     @tf.contrib.eager.defun
#     def upsample_wrap(self,inp):
#         return tf.map_fn(self.upsample, inp)
        
    def call(self, inputs):
#         return self.upsample_wrap(inputs)
        if self.data_format == 'channels_first':
            return tf.transpose(tf.image.resize_images(tf.transpose(inputs, (0,2,3,1)), tf.multiply(self.upsample_ratio, inputs.shape[2:]), align_corners=True, method=self.resize_method), (0,3,1,2))
        else:
            return tf.image.resize_images(inputs, tf.multiply(self.upsample_ratio, inputs.shape[1:-1]), align_corners=True, method=self.resize_method)
        

In [317]:
def poisson_matrix(m,n):
    '''
    Generates the matrix A to express the Poisson equation in the form Ax=b for an m-by-n grid
    
    Them matrix returned shall be (m-2)*(n-2)-by-(m-2)*(n-2) in size
    
    CURRENTLY ONLY WORKS FOR SQUARE DOMAINS!!!!
    '''
    m = m-2
    n = n-2
    
    D = np.zeros((m,m), dtype = np.float64)
    i,j = np.indices(D.shape)
    D[i==j] = 4.0
    D[i==j-1] = -1.0
    D[i==j+1] = -1.0
    
    S = -np.eye(D.shape[0], dtype = np.float64)
    
    P = np.zeros((m*n,m*n), dtype = np.float64)
    ind = np.arange(0,m*(n+1), m)
    
    for i in range(len(ind)-1):
        P[ind[i]:ind[i+1], ind[i]:ind[i+1]] = D
        try:
            P[ind[i+1]:ind[i+2], ind[i]:ind[i+1]] = S
        except:
            pass
        try:
            P[ind[i-1]:ind[i], ind[i]:ind[i+1]] = S
        except:
            pass
    return P

def generate_random_RHS(n, n_controlpts = None, n_outputpts = None, s = 5, domain = [0,1,0,1]):
    
    '''
    This function generates random smooth RHS 'functions' defined pointwise using bivariate splines. 
    n: no. of random RHSes to generate
    n_controlpts: no. of control pts of the spline. Smaller values lead to 'smoother' results
    n_outputpts: no. of gridpoints in each direction of the output 
    s: see parameter s in scipy.interpolate.RectBivariateSpline
    domain: [x_min, x_max, y_min, y_max]
    '''
    
    
    if isinstance(n, Iterable):
        n_controlpts = n[1]
        n_outputpts = n[2]
        try:
            s = n[3]
        except:
            pass
        try:
            domain = n[4]
        except:
            pass
        n = n[0]
    
    x = np.linspace(domain[0], domain[1], n_controlpts)
    y = np.linspace(domain[2], domain[3], n_controlpts)
    if n_controlpts != n_outputpts:
        x_out = np.linspace(domain[0], domain[1], n_outputpts)
        y_out = np.linspace(domain[2], domain[3], n_outputpts)
    else:
        x_out = x
        y_out = y
            
    out = []
    for i in range(n):
        spl = RectBivariateSpline(x,y,2*np.random.rand(len(x), len(y))-1, s=s)
        out.append(spl(x_out,y_out))
    return np.array(out)
    

def poisson_RHS(F, boundaries = None, h = None):
    '''
    Generates the RHS vector b of a discretized Poisson problem in the form Ax=b.
    h = grid spacing
    boundaries = dict containing entries 'top', 'bottom', 'right' and 'left' which correspond to the Dirichlet BCs at these boundaries. Each entry must be a vector of length m or n, where m and n are defined as in te function poisson_matrix
    F = an m by n matrix containing the RHS values of the Poisson equation
    
    (i.e. this function merely takes the BC information and the array from generate_random_RHS to provide the RHS for the matrix eq. form)
    '''
    
    if isinstance(F, Iterable):
        boundaries = F[1]
        h = F[2]
        F = F[0]
    
    F = -h**2 * F
    F[...,1:-1,1] = F[...,1:-1,1] + np.array(boundaries['top'])[1:-1]
    F[...,1:-1,-2] = F[...,1:-1,-2] + np.array(boundaries['bottom'])[1:-1]
    F[...,1,1:-1] = F[...,1,1:-1] + np.array(boundaries['left'])[1:-1]
    F[...,-2,1:-1] = F[...,-2,1:-1] + np.array(boundaries['right'])[1:-1]
    
    return F[...,1:-1,1:-1].reshape(list(F[...,1:-1,1:-1].shape[:-2]) + [np.prod(F[...,1:-1,1:-1].shape[-2:])])
 
def generate_dataset(batch_size, n, h, boundaries, n_batches = 1, rhs_range = [-1,1]):
    lhs = tf.constant(poisson_matrix(n,n), dtype=tf.float64)
    lhs_chol = tf.linalg.cholesky(lhs)
    
    def chol(r):
        return tf.linalg.cholesky_solve(lhs_chol, tf.transpose(tf.stack([r])))
    
    @tf.contrib.eager.defun
    def chol_solve(rhs_arr):
        return tf.map_fn(chol, rhs)
    
    #pdb.set_trace()
    pool = ThreadPool(n_batches)
    F = pool.map(generate_random_RHS, zip(itertools.repeat(batch_size, n_batches), itertools.repeat(10), itertools.repeat(n), itertools.repeat(5), itertools.repeat([0,n*h,0,n*h])))
    rhs = tf.concat(pool.map(poisson_RHS, zip(F, itertools.repeat(boundaries), itertools.repeat(h))), axis=0)
    
    soln = np.zeros((n_batches * batch_size, n, n), dtype = np.float64)
    soln[...,:,0] = boundaries['top']
    soln[...,:,-1] = boundaries['bottom']
    soln[...,0,:] = boundaries['left']
    soln[...,-1,:] = boundaries['right']
    soln[:,1:-1,1:-1] = tf.reshape(chol_solve(rhs), (n_batches * batch_size, n-2, n-2))
    #soln = chol_solve(rhs)
    
    #return tf.reshape(soln, (n_batches, batch_size, lhs_chol.shape[0])), tf.reshape(F, (n_batches, batch_size, F.shape[-2], F.shape[-1]))
    return tf.reshape(soln, (n_batches*batch_size, 1, soln.shape[-2], soln.shape[-1])), tf.reshape(tf.concat(F, axis = 0), (n_batches*batch_size, 1, F[0].shape[-2], F[0].shape[-1]))


In [332]:
def Lp_integral_norm(image_size, domain, n_quadpts = 10, quadpts_randomization = 0, p=2, dtype = tf.float32):
    '''
    This function generates a function that takes 2 function(s) evaluated on a 2D grid, the dimensions of which are stored in image_size, on a rectangular domain and evaluates the Lp norm of their difference using Gauss-Legendre quadrature
    
    image_size              :     tuple with 2 elements.
    domain                  :     tuple with 4 elements containing (xmin,xmax,ymin,ymax)
    n_quadpts               :     no of GL quadrature points
    quadpts_randomization   :     if set to a nonzero value, the dataset inputted to the generated func. will be split into 2*quadpts_randomization+1 pieces and the integral for the ith piece will be computed with n_quadpts + i * quadpts_randomization GL pts (where i = [n_quadpts-quadpts_randomization, n_quadpts+quadpts_randomization])
    p                       :     order of the Lp norm
    
    The shape of the inputs to the generated function must be (batch_size,channels,image_size[0],image_size[1])
    
    User inputs to the closure (generated function):
    y_true                  :     labels
    y_pred                  :     images
    
    '''
    
    #arrays to store values for different # of GL quad. pts. in accordance with quadpts_randomization. length of each: 2 * quadpts_randomization + 1
    interpolation_weights = [] #array that stores weights b_ii such that the interpolated value at x,y is b_ii*f(X_i, Y_j) within a rectangle bounded by [X_1,X_2] x [Y_1, Y_2] (shapes: (n_quadpts, n_quadpts, 4, 2))
    quadweights_list = [] #array that stores GL quad weights (shapes: (n_quadpts, n_quadpts))
    index_combinations_list = [] #array of tensors, where each tensor stores the indices for the 4 points between which every GL quad pt lies (shapes: (no of quadpts, no of quadpts, 4, 2))
    coords = np.array(np.meshgrid(np.linspace(domain[0], domain[1], image_size[0]),np.linspace(domain[2], domain[3], image_size[1]),indexing = 'xy'), dtype = np.float32).transpose((1,2,0)) #coordinates of each grid pt in the domain
    image_coords = [coords[0,:,0], coords[:,1,1]] #x and y coordinates separately
    c = np.array([np.array(0.5*(domain[1] - domain[0]),dtype=np.float64),np.array(0.5*(domain[3] - domain[2]),dtype=np.float32)]) #scaling coefficients - for handling domains other than [-1,1] x [-1,1]
    d = np.array([np.array(0.5*(domain[1] + domain[0]),dtype=np.float64),np.array(0.5*(domain[3] + domain[2]),dtype=np.float32)])
    for n in range(n_quadpts - quadpts_randomization, n_quadpts + quadpts_randomization+1): #loop over no of quadpts
        quadrature_x, quadrature_w = tuple([np.polynomial.legendre.leggauss(n)[i].astype(np.float64) for i in range(2)]) #quadrature weights and points
        quadpts = tf.constant(np.apply_along_axis(lambda x: x + d, 0, np.einsum('ijk,i->ijk',np.array(np.meshgrid(quadrature_x,quadrature_x,indexing = 'xy')),c)).transpose((1,2,0)),dtype = tf.float32)
        quadweights = tf.reduce_prod(c)*tf.tensordot(quadrature_w,quadrature_w,axes = 0)
        indices = [[],[]] #indices between each quadrature point lies - indices[0] is in x-dir and indices[1] is in the y-dir
        quad_coords = [quadpts[0,:,0], quadpts[:,1,1]] #x and y coordinates of each quad pt respectively
        #find the indices of coords between which every quad. pt. lies
        for i in range(len(indices)):
            j=0
            #does not work if more than 2 quad pts fall within 1 cell - fix later
            while len(indices[i]) < quadpts.shape[0] and j<image_coords[i].shape[0]:
                try:
                    if abs(float(quad_coords[i][len(indices[i])] - image_coords[i][j])) == float(min(abs(quad_coords[i][len(indices[i])] - image_coords[i][j-1]), abs(quad_coords[i][len(indices[i])] - image_coords[i][j]), abs(quad_coords[i][len(indices[i])] - image_coords[i][j+1]))):
                        if quad_coords[i][len(indices[i])] - image_coords[i][j] < 0:
                            indices[i].append((j-1,j))
                        else:
                            indices[i].append((j,j+1))
                except:
                    if abs(float(quad_coords[i][len(indices[i])] - image_coords[i][j])) == float(min(abs(quad_coords[i][len(indices[i])] - image_coords[i][j-1]), abs(quad_coords[i][len(indices[i])] - image_coords[i][j]))):
                        indices[i].append((j-1,j))
                j+=1
        
        index_combinations = np.zeros((quadpts.shape[0], quadpts.shape[1], 4 , 2), dtype = np.int32) #array storing the 4 index combinations on the original grid which surround each quad. pt.
        corners = np.zeros((quadpts.shape[0], quadpts.shape[1], 2 , 2), dtype = np.int32) #array storing the lower left corner and the upper right corner of each box stored in index_combinations. effectively this will contain [[xmin,ymin],[xmax,ymax]] for the rectangle around each quad pt.
        s=np.array(indices)
        for i in range(n):
            for j in range(n):
                index_combinations[i,j,:,:] = np.array(list(itertools.product(np.array(s)[0,i,:],np.array(s)[1,j,:])))
        for i in range(n):
            for j in range(n):
                corners[i,j,:,:] = np.array([s[0,i,:],s[1,j,:]])
        corners = corners.transpose((0,1,3,2))
        corner_coords = tf.gather_nd(tf.transpose(coords,(1,0,2)),corners)
        
        #compute the coefficients [b_11,b_12,b_21,b_22]
        #steps:
        #1. compute transpose(invert(array([[1,xmin,ymin,xmin*ymin],[1,xmin,ymax,xmin*ymax],[1,xmax,ymin,xmax*ymin],[1,xmax,ymax,xmax*ymax]]))) for the rectangle around each quad pt.
        #2. compute array([1,x_quadpt, y_quadpt, x_quadpt*y_quadpt]) for each quadpt
        #3. multiply the result of 1 and 2 for each quad pt.
        interpolation_matrix = np.ones((n,n,4,4))
        interpolation_matrix[:,:,0:2,1] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,0:2,1],corner_coords[:,:,0,0])
        interpolation_matrix[:,:,2:,1] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,2:,1],corner_coords[:,:,1,0])
        interpolation_matrix[:,:,(0,2),2] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,(0,2),2],corner_coords[:,:,0,1])
        interpolation_matrix[:,:,(1,3),2] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,(1,3),2], corner_coords[:,:,1,1])
        interpolation_matrix[:,:,:,3] *= np.multiply(interpolation_matrix[:,:,:,1], interpolation_matrix[:,:,:,2])
        interpolation_matrix = tf.transpose(tf.linalg.inv(interpolation_matrix), (0,1,3,2))
        q = np.ones((n,n,4))
        q[:,:,1] = tf.transpose(quadpts[:,:,0])
        q[:,:,2] = tf.transpose(quadpts[:,:,1])
        q[:,:,3] = np.multiply(q[:,:,1],q[:,:,2])
        
        b = tf.einsum('ijkl, ijl->ijk', tf.constant(interpolation_matrix), tf.constant(q))
        
        #store results for the closure
        quadweights_list.append(tf.cast(quadweights,dtype))
        index_combinations_list.append(index_combinations)
        interpolation_weights.append(tf.cast(b, dtype))
    
    @tfe.defun
    def Lp_integrate_batch(inp):
        '''
        Helper function to facilitate quad pt randomization
        
        Given the bilinear interp. weights b, GL quad. weights w, Lp norm order p and the indices bounding each quad pt ind, computes the Lp norm for each channel/batch element
        '''
        #unpack values
        data = tf.transpose(inp[0], (2,3,1,0))
        b = inp[1]
        w = inp[2]
        ind = inp[3]
        p = inp[4]
        #print(w)
        #get the points from the image to perform interpolation on
        interp_pts = tf.squeeze(tf.gather_nd(data, ind))
        #pdb.set_trace()
        #multiply image values with the weights b ti interpolate original image onto the GL quadrature points
        values_at_quad_pts = tf.einsum('ijkl, ijk->ijl', interp_pts, b)
        #pdb.set_trace()
        #compute Lp norm
        return tf.pow(tf.reduce_sum(tf.einsum('ij,ijk->ijk',w,tf.pow(values_at_quad_pts, p)), axis = (0,1)), 1/p)
    
    @tfe.defun
    def Lp_integrate(y_true,y_pred, b=interpolation_weights, w=quadweights_list, ind=index_combinations_list, p=p):
        '''
        Split the batch, pack with the appropriate parameters and obtain the integrals
        '''
        #pool = ThreadPool(len(b))
        return tf.concat(list(map(Lp_integrate_batch, zip(tf.split(y_true-y_pred, len(b)), itertools.cycle(b), itertools.cycle(w), itertools.cycle(ind), itertools.repeat(p)))), 0)
        
    return Lp_integrate
    

In [319]:
ntest = 64
h = 0.05
boundary_top = Boundary1D('Dirichlet', [(0,ntest*h),(ntest*h,ntest*h)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)
boundary_right = Boundary1D('Dirichlet', [(ntest*h,ntest*h),(ntest*h,0)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)
boundary_bottom = Boundary1D('Dirichlet', [(ntest*h,0),(0,0)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)
boundary_left = Boundary1D('Dirichlet', [(0,0),(0,ntest*h)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)

In [ ]:
import time
t0 = time.time()
soln,F = generate_dataset(batch_size=2000, n = ntest, h = h, n_batches=20, boundaries={'top': boundary_top.RHS_evaluate(np.linspace(boundary_top.t.min(),boundary_top.t.max(),ntest)), 'right': boundary_right.RHS_evaluate(np.linspace(boundary_right.t.min(),boundary_right.t.max(),ntest)), 'bottom': boundary_bottom.RHS_evaluate(np.linspace(boundary_bottom.t.min(),boundary_bottom.t.max(),ntest)), 'left': boundary_left.RHS_evaluate(np.linspace(boundary_left.t.min(),boundary_left.t.max(),ntest))})
t1 = time.time()
print('Generation of training data took ' + str(t1-t0) + ' seconds')
with h5py.File('dataset8.h5', 'w') as hf:
    hf.create_dataset('soln', data=soln)
    hf.create_dataset('F', data=F)

In [320]:
for i in range(0,5):
    with h5py.File('dataset' + str(i) + '.h5', 'r') as hf:
        F = np.array(hf.get('F'), dtype = np.float64)
        soln = np.array(hf.get('soln'), dtype = np.float64)
        try:
            train_data = train_data.concatenate(tf.data.Dataset.from_tensor_slices((F,soln)))
        except:
            train_data = tf.data.Dataset.from_tensor_slices((F,soln))

In [321]:
for i in range(6,7):
    with h5py.File('dataset' + str(i) + '.h5', 'r') as hf:
        F = np.array(hf.get('F'), dtype = np.float64)
        soln = np.array(hf.get('soln'), dtype = np.float64)
        try:
            valid_data = valid_data.concatenate(tf.data.Dataset.from_tensor_slices((F,soln)))
        except:
            valid_data = tf.data.Dataset.from_tensor_slices((F,soln))

In [ ]:
# with h5py.File('dataset' + str(4) + '.h5', 'r') as hf:
#         F_valid = np.array(hf.get('F'), dtype = np.float64)
#         soln_valid = np.array(hf.get('soln'), dtype = np.float64)

In [341]:
shuffle_size = 100000
batch_size = 3000
train_data = train_data.shuffle(shuffle_size).batch(batch_size)
valid_data = valid_data.shuffle(shuffle_size).batch(batch_size)
del F, soln

In [323]:
input_0 = Input(shape=(1,ntest,ntest,))
conv_1_0 = Conv2D(filters = 5, kernel_size = 3, activation=tf.nn.relu, data_format='channels_first', padding='same')(input_0)

conv_2_0 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format='channels_first', padding='same')(conv_1_0)
pool_2_0 = AveragePooling2D(data_format = 'channels_first')(conv_2_0)
pool_2_1 = AveragePooling2D(data_format = 'channels_first')(pool_2_0)

conv_3_0 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format = 'channels_first', padding='same')(conv_2_0)
conv_3_1 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format = 'channels_first', padding='same')(pool_2_0)
conv_3_2 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format = 'channels_first', padding='same')(pool_2_1)
#transposeconv_3_1 = Conv2DTranspose(filters = 8, kernel_size = 3, strides = 2, activation = tf.nn.relu, data_format = 'channels_first', padding='same')(conv_3_1)
#transposeconv_3_2 = Conv2DTranspose(filters = 8, kernel_size = 3, strides = 4, activation = tf.nn.relu, data_format = 'channels_first', padding='same')(conv_3_2)
#upsample_3_1 = UpSampling2D(size=(2,2), data_format = 'channels_first', interpolation = 'bilinear')(conv_3_1)
#upsample_3_2= UpSampling2D(size=(4,4), data_format = 'channels_first', interpolation = 'bilinear')(conv_3_2)
upsample_3_1 = Upsample()(conv_3_1)
upsample_3_2 = Upsample(4)(conv_3_2)
#merge_3_0 = Add()([conv_3_0, transposeconv_3_1, transposeconv_3_2])
merge_3_0 = Add()([conv_3_0, upsample_3_1, upsample_3_2])

conv_4_0 = Conv2D(filters = 8, kernel_size = 1, activation = tf.nn.relu, data_format = 'channels_first', padding='same')(merge_3_0)

conv_5_0 = Conv2D(filters = 1, kernel_size = 1, activation = 'linear', data_format = 'channels_first', padding='same')(conv_4_0)
final_activation = tf.keras.layers.PReLU()(conv_5_0)
a = Model(input_0, final_activation)

In [342]:
a.compile(optimizer = tf.train.AdamOptimizer(learning_rate=1e-2), loss = Lp_integral_norm((ntest,ntest), [0,ntest*h,0,ntest*h], n_quadpts=8, quadpts_randomization=2, p=2), metrics = ['accuracy'])
a.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 64, 64)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 5, 64, 64)    50          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 8, 64, 64)    368         conv2d[0][0]                     
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 8, 32, 32)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
average_po

In [343]:
a.fit(train_data, steps_per_epoch=5, epochs=10, validation_data=valid_data, validation_steps=2)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[1,1,100,2000,1,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Cast] name: Cast/

In [ ]:
# a.save_weights('model.h5')

In [ ]:
p = np.random.randint(0,F_valid.shape[0])
y, x = np.meshgrid(np.linspace(0, ntest*h, ntest), np.linspace(0, ntest*h, ntest))
z = a.predict(tf.expand_dims(F[p,...], axis=0))[0,0,...]
#z = soln[p,0,...]
#z = generate_random_RHS(10, n_controlpts=10, n_outputpts=64)[4,:,:]
#z = a.predict(tf.expand_dims(F_valid[p,...], axis=0))[0,0,...] - soln_valid[p,0,...]
z_min, z_max = -np.abs(z).max(), np.abs(z).max()
fig, ax = plt.subplots()
c = ax.pcolormesh(x, y, z, cmap='RdBu', vmin=z_min, vmax=z_max)
ax.set_title('pcolormesh')
ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
#soln_valid,F_valid = generate_dataset(batch_size=50, n = ntest, h = h, n_batches=1, boundaries={'top': boundary_top.RHS_evaluate(np.linspace(boundary_top.t.min(),boundary_top.t.max(),ntest)), 'right': boundary_right.RHS_evaluate(np.linspace(boundary_right.t.min(),boundary_right.t.max(),ntest)), 'bottom': boundary_bottom.RHS_evaluate(np.linspace(boundary_bottom.t.min(),boundary_bottom.t.max(),ntest)), 'left': boundary_left.RHS_evaluate(np.linspace(boundary_left.t.min(),boundary_left.t.max(),ntest))})
#p = np.random.randint(0,F_valid.shape[0])
y, x = np.meshgrid(np.linspace(0, ntest*h, ntest), np.linspace(0, ntest*h, ntest))
z = F[p,0,...]
#z = soln[p,0,...]
z_min, z_max = -np.abs(z).max(), np.abs(z).max()
fig, ax = plt.subplots()
c = ax.pcolormesh(x, y, z, cmap='RdBu', vmin=z_min, vmax=z_max)
ax.set_title('pcolormesh')
ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
i = np.random.randint(1,ntest-2)
j = np.random.randint(1,ntest-2)
p = np.random.randint(0,soln_valid.shape[0]-1)
print((-4 * soln_valid[p,0,i,j] + soln_valid[p,0,i,j-1] + soln_valid[p,0,i-1,j] + soln_valid[p,0,i+1,j] + soln_valid[p,0,i,j+1])/(h**2))
example_soln = a.predict(tf.expand_dims(F_valid[p,...], axis=0))
print((-4 * example_soln[0,0,i,j] + example_soln[0,0,i,j-1] + example_soln[0,0,i-1,j] + example_soln[0,0,i+1,j] + example_soln[0,0,i,j+1])/(h**2))
print(F_valid[p,0,i,j])
print(soln_valid[p,0,i,j])

In [ ]:
X,Y = np.meshgrid(np.linspace(0,1,20),np.linspace(0,1,20))
Z_m = np.array([[np.sin(np.pi*(X+Y)), np.cos(np.pi * (X+Y))] , [np.exp(-X**2 * Y**2), np.tan(np.multiply(X,Y))]])
#Z_m = tf.expand_dims(Z, axis = 0)
ur = 3
b = BilinearUpsample(ur)(input_0)
model1 = Model(input_0, b)

In [ ]:
Z_m.shape

In [ ]:
X0, Y0 = np.meshgrid(np.linspace(0,1,X.shape[0]*ur),np.linspace(0,1,X.shape[0]*ur))
fig, ax = plt.subplots()
c = ax.pcolormesh(X0, Y0, model1(Z_m)[0,0,...], cmap='RdBu')

In [ ]:
fig, ax = plt.subplots()
c = ax.pcolormesh(X, Y, Z_m[0,0,...], cmap='RdBu')

In [ ]:
np.mean(np.abs(a.predict(F_valid) - soln_valid))

In [ ]:
timeit a.predict(tf.expand_dims(F_valid[0,...], axis=0))

In [ ]:
timeit a.predict(F_valid[0:1000,...])

In [304]:
nx = 110
ny = 73
X,Y = np.meshgrid(np.linspace(0,1,nx), np.linspace(0,2,ny), indexing = 'ij')
f = tf.expand_dims(np.array([X + Y,X*Y, X-Y, X**2 + Y**2, X**2 - Y**2, X-2*Y]), axis = 1)

In [312]:
g = Lp_integral_norm((nx,ny), [0,1,0,2], n_quadpts=20, quadpts_randomization=1, p=2)

In [313]:
g(f, np.zeros((6,1,nx,ny), dtype = np.float64))

<tf.Tensor: id=2760583, shape=(6,), dtype=float64, numpy=
array([2.30940108, 0.94280905, 1.15470054, 2.92895012, 2.24112116,
       2.70801284])>

In [303]:
timeit g(tf.random.uniform((3000,1,nx,ny), dtype=tf.float64), tf.random.uniform((3000,1,nx,ny), dtype = tf.float64))

9.48 ms ± 57.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
